Get raw data using nba_api #https://github.com/swar/nba_api

In [1]:
#IMPORTING MAIN PACKAGES
import pandas as pd 
import numpy as np
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

PLAYERS INFORMATION

In [2]:
#IMPORTING PLAYERS INFO USING NBA_API
from nba_api.stats.endpoints import commonallplayers

In [4]:
#IMPORTING PLAYERS INFO
all_players = commonallplayers.CommonAllPlayers().get_data_frames()[0]

In [5]:
# CHANGE DATA TYPE
all_players['TO_YEAR'] = all_players['TO_YEAR'].astype('int64')

In [6]:
# FILTER ONLY SEASONS FROM 1990 TO 2021
all_players = all_players[all_players['TO_YEAR']>=1990]

In [16]:
# IMPORTING PLAYERS STATS USING NBA_API
from nba_api.stats.endpoints import playercareerstats
import time

In [17]:
# ESTRACT ALL IDs
players_ID = all_players['PERSON_ID']

In [18]:
# INITIATE AN DATAFRAME FOR PLAYERS STATS
players_stats = pd.DataFrame()
# SAVE FAIL QUERIES
error_log = []

In [19]:
# EXTRACTING STATS
for ID in tqdm(players_ID):
    try:
        time.sleep(1) # avoid too many queries submitted at the same time
        career = playercareerstats.PlayerCareerStats(player_id=ID)
        player_career = career.get_data_frames()[0]
        players_stats = pd.concat([players_stats,player_career],axis=0,ignore_index=True)
    except:
        error_log.append(ID)

100%|██████████| 2217/2217 [8:15:17<00:00, 13.40s/it]      


In [20]:
# RE-QUERING MISSING IDs
for ID in tqdm(error_log):
    try:
        time.sleep(1)
        career = playercareerstats.PlayerCareerStats(player_id=ID)
        player_career = career.get_data_frames()[0]
        players_stats = pd.concat([players_stats,player_career],axis=0,ignore_index=True)
    except:
        error_log.append(ID)

100%|██████████| 5/5 [00:14<00:00,  2.87s/it]


In [21]:
#SAVING DATA IN AN DATASET .PKL
with open("player_stats.pkl",'wb') as f:
    pickle.dump(players_stats,f)




TEAMS INFO

In [22]:
#IMPORTING TEAMS STATS AND INFO USING NBA_API
from nba_api.stats.endpoints import teamyearbyyearstats
from nba_api.stats.static import teams

In [23]:
#IMPORTING TEAMS STATS AND INFO USING NBA_API
nba_teams = teams.get_teams()

In [24]:
#EXTRACTING TEAMS STATS
teams_stats = pd.DataFrame()
error_teams =[]
for i in tqdm(nba_teams):
    try:
        time.sleep(1)
        team = teamyearbyyearstats.TeamYearByYearStats(team_id=i['id'])
        team_data = team.get_data_frames()[0]
        teams_stats = pd.concat([teams_stats,team_data],axis=0,ignore_index=True)
    except:
        error_teams.append(i)

100%|██████████| 30/30 [01:00<00:00,  2.03s/it]


In [25]:
#SAVING DATA IN AN DATASET .PKL
with open("teams_stats.pkl","wb") as f:
    pickle.dump(teams_stats,f)


AWARDS INFO

In [27]:
#IMPORTING AWARDS INFO USING NBA_API
from nba_api.stats.endpoints import playerawards

In [28]:
#EXTRACTING TEAMS STATS
players_awards = pd.DataFrame()
error_awards = []
for ID in tqdm(players_ID):
    try:
        time.sleep(1)
        award = playerawards.PlayerAwards(player_id=ID)
        award.get_data_frames()[0]
        players_awards = pd.concat([players_awards,award.get_data_frames()[0]],axis=0,ignore_index=True)
    except:
        error_awards.append(ID)

 96%|█████████▌| 2132/2217 [53:01<02:03,  1.45s/it] 

In [ ]:
#RE-QUERING MISSING IDS
for ID in tqdm(error_awards):
    time.sleep(1)
    award = playerawards.PlayerAwards(player_id=ID)
    award.get_data_frames()[0]
    players_awards = pd.concat([players_awards,award.get_data_frames()[0]],axis=0,ignore_index=True)

0it [00:00, ?it/s]


In [ ]:
#SAVING DATA IN AN DATASET .PKL
with open("players_awards.pkl","wb") as f:
    pickle.dump(players_awards,f)